<a href="https://colab.research.google.com/github/PatoJosefo/API-NEXUS/blob/main/colab_unificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output,Javascript
from google.colab import drive
import seaborn as sns
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive montado com sucesso.")
except Exception as e:
    print(f"Erro ao montar Google Drive: {e}")
    print("Por favor, verifique se você autorizou o acesso e se o caminho do Drive está correto.")

DRIVE_BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/API1S/API1S/base_dados_por_municipio/"
print(f"Usando caminho base para os dados: {DRIVE_BASE_PATH}")

In [ ]:
municipios_file = f"{DRIVE_BASE_PATH}UF_MUN.csv"
ncm_sh_file = f"{DRIVE_BASE_PATH}NCM_SH.csv"

try:
    print(f"Tentando carregar: {municipios_file}")
    municipios_df = pd.read_csv(
        municipios_file,
        delimiter=';',
        encoding='latin-1'
    )
    municipios_df.columns = municipios_df.columns.str.strip()
    print(f"Carregado UF_MUN.csv ({len(municipios_df)} rows)")

    print(f"Tentando carregar: {ncm_sh_file}")
    ncm_sh_df = pd.read_csv(
        ncm_sh_file,
        delimiter=';',
        encoding='latin-1'
    )
    ncm_sh_df.columns = ncm_sh_df.columns.str.strip()
    print(f"Carregado NCM_SH.csv ({len(ncm_sh_df)} rows)")

    municipios_df['CO_MUN_GEO'] = pd.to_numeric(municipios_df['CO_MUN_GEO'], errors='coerce')
    rows_before = len(municipios_df)
    municipios_df.dropna(subset=['CO_MUN_GEO'], inplace=True) # Remover linhas onde CO_MUN_GEO não é numérico
    print(f"  Municípios: Removidas {rows_before - len(municipios_df)} linhas com CO_MUN_GEO inválido.")
    municipios_df['CO_MUN_GEO'] = municipios_df['CO_MUN_GEO'].astype(int)

    ncm_sh_df['CO_SH4'] = pd.to_numeric(ncm_sh_df['CO_SH4'], errors='coerce')
    rows_before = len(ncm_sh_df)
    ncm_sh_df.dropna(subset=['CO_SH4'], inplace=True) # Remover linhas onde CO_SH4 não é numérico
    print(f"  NCM_SH: Removidas {rows_before - len(ncm_sh_df)} linhas com CO_SH4 inválido.")
    ncm_sh_df['CO_SH4'] = ncm_sh_df['CO_SH4'].astype(int)

except FileNotFoundError as e:
    print(f"Erro ao carregar arquivo auxiliar: {e}")
    print(f"Por favor, verifique se '{e.filename}' existe no caminho especificado: {DRIVE_BASE_PATH}")
    print(f"Verifique o DRIVE_BASE_PATH na Célula 2 e os nomes dos arquivos.")
    municipios_df = pd.DataFrame()
    ncm_sh_df = pd.DataFrame()
except Exception as e:
    print(f"Ocorreu um erro inesperado ao carregar dados auxiliares: {e}")
    municipios_df = pd.DataFrame()
    ncm_sh_df = pd.DataFrame()

In [ ]:
def carregar_dados_anuais(anos_lista):
    dados = {}
    print(f"Iniciando carregamento de dados para os anos: {anos_lista}...")

    if 'municipios_df' not in globals() or municipios_df.empty or \
       'ncm_sh_df' not in globals() or ncm_sh_df.empty:
         print("Dados auxiliares (municipios_df ou ncm_sh_df) não foram carregados corretamente ou estão vazios.")
         print("Certifique-se de que a Célula 3 foi executada sem erros antes de rodar esta célula")
         return None


    for ano in anos_lista:
        file_path = f"{DRIVE_BASE_PATH}EXP_{ano}_MUN.csv"
        print(f"Tentando carregar: {file_path}")
        try:
            df = pd.read_csv(
                file_path,
                delimiter=';',
                dtype={'KG_LIQUIDO': str, 'VL_FOB': str, 'CO_MUN': str, 'SH4': str},
                encoding='latin-1'
            )
            df.columns = df.columns.str.strip()
            print(f"Carregado {ano}, processando...")

            df_sp = pd.DataFrame()
            if 'SG_UF_MUN' in df.columns:
                 df_sp = df[df['SG_UF_MUN'].str.contains('SP', case=False, na=False)].copy()
                 print(f"  Filtrado para SP usando 'SG_UF_MUN': {len(df_sp)} linhas encontradas.")
            elif 'CO_MUN' in df.columns and 'municipios_df' in globals() and not municipios_df.empty:
                 print("'SG_UF_MUN' não encontrado ou inutilizável. Filtrando SP baseado em 'CO_MUN' usando dados auxiliares.")
                 sp_codes = municipios_df[municipios_df['SG_UF'] == 'SP']['CO_MUN_GEO'].unique()
                 df['CO_MUN_temp'] = pd.to_numeric(df['CO_MUN'], errors='coerce')
                 df_filtered_numeric = df.dropna(subset=['CO_MUN_temp']).copy()
                 df_filtered_numeric['CO_MUN_temp'] = df_filtered_numeric['CO_MUN_temp'].astype(int)
                 df_sp = df_filtered_numeric[df_filtered_numeric['CO_MUN_temp'].isin(sp_codes)].copy()
                 df_sp = df_sp.drop(columns=['CO_MUN_temp'])
                 print(f"Filtrado para SP usando 'CO_MUN': {len(df_sp)} linhas encontradas.")
            else:
                 print("Motivo: coluna 'SG_UF_MUN' ausente ou inutilizável, E (coluna 'CO_MUN' ausente ou 'municipios_df' auxiliar indisponível/vazio).")
                 print("Prosseguindo com dados não filtrados para este ano, que podem incluir localizações fora de SP.")
                 df_sp = df.copy()

            if df_sp.empty:
                print(f"Nenhum dado de SP encontrado (ou filtrado) para {ano}. Pulando processamento adicional para este ano.")
                continue

            df_sp['KG_LIQUIDO'] = pd.to_numeric(
                df_sp['KG_LIQUIDO'].str.replace(',', '.', regex=False), errors='coerce'
            )
            df_sp['VL_FOB'] = pd.to_numeric(
                df_sp['VL_FOB'].str.replace(',', '.', regex=False), errors='coerce'
            )
            df_sp['CO_MUN'] = pd.to_numeric(df_sp['CO_MUN'], errors='coerce')
            df_sp['SH4'] = pd.to_numeric(df_sp['SH4'], errors='coerce')

            essential_cols = ['CO_MUN', 'SH4', 'KG_LIQUIDO', 'VL_FOB']
            initial_rows = len(df_sp)
            df_sp.dropna(subset=essential_cols, inplace=True)
            dropped_rows = initial_rows - len(df_sp)
            if dropped_rows > 0:
                print(f"Removidas {dropped_rows} linhas devido a valores essenciais ausentes/inválidos em: {', '.join(essential_cols)}.")

            if not df_sp.empty:
                df_sp['CO_MUN'] = df_sp['CO_MUN'].astype(int)
                df_sp['SH4'] = df_sp['SH4'].astype(int)
            else:
                print(f"Nenhum dado válido restou para {ano} após limpeza das colunas essenciais.")
                continue

            if not df_sp.empty:
                 dados[ano] = df_sp
                 print(f"Dados para {ano} processados e armazenados com sucesso. Formato final: {df_sp.shape}")

        except FileNotFoundError:
            print(f"--- AVISO ---: Arquivo não encontrado para o ano {ano} em {file_path}. Pulando este ano.")
        except Exception as e:
            print(f"--- ERRO ---: Ocorreu um erro inesperado ao carregar ou processar dados para o ano {ano}: {e}")

    if not dados:
        print("Por favor, verifique os caminhos dos arquivos, existência dos arquivos e possíveis erros nos logs acima.")
    else:
        print(f"\nCarregamento de dados anuais concluído. Anos carregados com sucesso: {list(dados.keys())}")

    return dados

In [ ]:
anos_para_carregar = ['2019', '2020', '2021', '2022', '2023', '2024']

dados_ano = carregar_dados_anuais(anos_para_carregar)

municipios_sp_list = ['-- N/A --']
municipios_sp = pd.DataFrame()

if dados_ano and 'municipios_df' in globals() and not municipios_df.empty:
    print("\nPreparando lista de municípios de SP para dropdowns...")
    all_sp_mun_codes = set()
    for ano, df_ano in dados_ano.items():
        if df_ano is not None and not df_ano.empty and 'CO_MUN' in df_ano.columns:
             all_sp_mun_codes.update(df_ano['CO_MUN'].unique())

    if all_sp_mun_codes:
        print(f"Encontrados {len(all_sp_mun_codes)} códigos CO_MUN únicos de SP nos dados carregados.")
        try:
            municipios_df['CO_MUN_GEO'] = municipios_df['CO_MUN_GEO'].astype(int)
            municipios_sp = municipios_df[
                (municipios_df['CO_MUN_GEO'].isin(all_sp_mun_codes)) &
                (municipios_df['SG_UF'] == 'SP')
            ].copy()
            if not municipios_sp.empty:
                 municipios_sp_list = sorted(municipios_sp['NO_MUN'].astype(str).unique().tolist())
                 print(f" Lista de {len(municipios_sp_list)} nomes únicos de municípios de SP encontrados nos dados criada com sucesso.")
                 if not municipios_sp_list:
                      print(" Aviso: A filtragem resultou em uma lista vazia, embora o subconjunto SP de municipios_df não estivesse vazio. Verifique a coluna 'NO_MUN'.")
                      municipios_sp_list = ['-- N/A --']
            else:
                 print("Aviso: Nenhum município dos dados carregados correspondeu a entradas de SP no arquivo auxiliar")
                 municipios_sp_list = ['-- N/A --']

        except Exception as e:
            print(f"Erro durante a filtragem/criação da lista de municípios de SP: {e}")
            municipios_sp_list = ['-- N/A --']
            municipios_sp = pd.DataFrame()

    else:
        print(" Aviso: Nenhum código de município de SP foi extraído dos dados anuais carregados. Não é possível criar lista de municípios.")
        municipios_sp_list = ['-- N/A --']
        municipios_sp = pd.DataFrame(columns=municipios_df.columns if 'municipios_df' in globals() else [])

else:
    if not dados_ano:
        print("Dados anuais ('dados_ano') não puderam ser carregados.")
    if 'municipios_df' not in globals() or municipios_df.empty:
        print("Dados auxiliares ('municipios_df') não estão disponíveis ou estão vazios.")
    print("Não é possível preparar lista de municípios de SP. Dropdowns serão limitados.")
    municipios_sp_list = ['-- N/A --']
    municipios_sp = pd.DataFrame(columns=municipios_df.columns if 'municipios_df' in globals() else [])

print(f"\nLista Final de Municípios de SP para Dropdowns (primeiros 20): {municipios_sp_list[:20]}")
if municipios_sp_list == ['-- N/A --']:
    print("-> Dropdowns mostrarão N/A. Por favor, verifique as etapas de carregamento de dados (Células 2-5).")

In [ ]:
print("Definindo widgets...")

valid_anos = list(dados_ano.keys()) if dados_ano else ['Select Year']
valid_municipios = municipios_sp_list if municipios_sp_list and municipios_sp_list[0] != '-- N/A --' else ['Select Municipality']

default_ano = valid_anos[0] if valid_anos and valid_anos[0] != 'Select Year' else None
default_municipio1 = valid_municipios[0] if valid_municipios and valid_municipios[0] != 'Select Municipality' else None
default_municipio2 = valid_municipios[1] if len(valid_municipios) > 1 and valid_municipios[0] != 'Select Municipality' else default_municipio1
ano_dropdown = widgets.Dropdown(
    options=valid_anos,
    value=default_ano,
    description='Ano:',
    disabled=not default_ano,
    style={'description_width': 'initial'}
)

municipio_dropdown = widgets.Dropdown(
    options=valid_municipios,
    value=default_municipio1,
    description='Município:',
    disabled=not default_municipio1,
    style={'description_width': 'initial'}
)

metrica_dropdown = widgets.Dropdown(
    options=['Quilogramas', 'Valor Agregado'],
    value='Quilogramas',
    description='Métrica:',
    style={'description_width': 'initial'}
)

top5_municipios_button = widgets.Button(
    description='Top 5 Municípios Exportadores (SP)',
    button_style='info',
    tooltip='Mostrar os 5 municípios de SP que mais exportaram no ano selecionado, pela métrica escolhida.',
    icon='list-ol',
    disabled=not default_ano
)

municipio1_dropdown = widgets.Dropdown(
    options=valid_municipios,
    value=default_municipio1,
    description='Comparar Município 1:',
    disabled=not default_municipio1,
    style={'description_width': 'initial'}
)

municipio2_dropdown = widgets.Dropdown(
    options=valid_municipios,
    value=default_municipio2,
    description='Comparar Município 2:',
    disabled=not default_municipio2,
    style={'description_width': 'initial'}
)

compare_button = widgets.Button(
    description='Comparar Top 5 Produtos',
    button_style='success',
    tooltip='Comparar os top 5 produtos (pela métrica) dos dois municípios selecionados.',
    icon='exchange-alt',
    disabled=not default_municipio1 or not default_municipio2
)

output_area = widgets.Output(layout={'border': '1px solid black', 'padding': '10px'})

print("Widgets definidos com sucesso.")

In [ ]:
def get_co_mun(nome_municipio):
    if 'municipios_sp' not in globals() or municipios_sp.empty:
         print(f"Aviso: DataFrame 'municipios_sp' não está disponível ou está vazio. Não é possível procurar CO_MUN para '{nome_municipio}'.")
         return None

    try:

        municipios_sp['NO_MUN'] = municipios_sp['NO_MUN'].astype(str)
        nome_municipio_str = str(nome_municipio)

        result = municipios_sp[municipios_sp['NO_MUN'] == nome_municipio_str]['CO_MUN_GEO']

        if not result.empty:
            return int(result.iloc[0])
        else:
            print(f"Aviso: Código CO_MUN não encontrado para o município '{nome_municipio_str}' na lista filtrada 'municipios_sp'.")
            return Non
    except Exception as e:
        print(f"Erro durante a pesquisa de CO_MUN para '{nome_municipio}': {e}")
        return None

def calcular_top_produtos(df_ano, co_mun, metrica):

    if df_ano is None or df_ano.empty:
        return pd.DataFrame()

    if co_mun is None:
        print("Aviso: CO_MUN inválido (None) fornecido para calcular_top_produtos.")
        return pd.DataFrame()

    if 'ncm_sh_df' not in globals() or ncm_sh_df.empty:
        print("Error: ncm_sh_df (descrições de produtos) não carregado. Não é possível calcular os produtos principais.")
        return pd.DataFrame()

    df_mun = df_ano[df_ano['CO_MUN'] == co_mun].copy()

    if df_mun.empty:
        return pd.DataFrame()

    calculated_metric_col = None
    df_metricas = pd.DataFrame()

    if metrica == 'Quilogramas':
        coluna_orig = 'KG_LIQUIDO'
        if coluna_orig not in df_mun.columns:
             print(f"Erro: Coluna necessária '{coluna_orig}' não encontrada nos dados para CO_MUN {co_mun}.")
             return pd.DataFrame()
        df_metricas = df_mun.groupby('SH4')[coluna_orig].sum().reset_index()
        calculated_metric_col = coluna_orig

    elif metrica == 'Valor Agregado':
        if not all(col in df_mun.columns for col in ['VL_FOB', 'KG_LIQUIDO']):
             print(f"Erro: Colunas necessárias 'VL_FOB' ou 'KG_LIQUIDO' não encontradas para CO_MUN {co_mun}.")
             return pd.DataFrame()

        df_metricas_agg = df_mun.groupby('SH4').agg(
            Total_FOB=('VL_FOB', 'sum'),
            Total_KG=('KG_LIQUIDO', 'sum')
        ).reset_index()

        epsilon = 1e-9
        df_metricas = df_metricas_agg[df_metricas_agg['Total_KG'] > epsilon].copy()

        if df_metricas.empty:
            return pd.DataFrame()

        df_metricas['Valor_Agregado'] = df_metricas['Total_FOB'] / df_metricas['Total_KG']
        calculated_metric_col = 'Valor_Agregado'
    else:
        print(f"Erro: Métrica desconhecida '{metrica}' fornecida para calcular_top_produtos.")
        return pd.DataFrame()


    if calculated_metric_col is None or calculated_metric_col not in df_metricas.columns:
         print(f"Erro: Coluna de métrica '{calculated_metric_col}' não foi calculada corretamente ou está ausente para CO_MUN {co_mun}.")
         return pd.DataFrame()

    if df_metricas.empty:
        return pd.DataFrame()

    top5 = df_metricas.nlargest(5, calculated_metric_col)

    try:
        top5['SH4'] = top5['SH4'].astype(int)
        if 'CO_SH4' not in ncm_sh_df.columns:
            print("Erro: Coluna 'CO_SH4' ausente em ncm_sh_df. Não é possível mesclar nomes de produtos.")
            top5['NO_SH4_POR'] = 'Erro no Merge (Sem CO_SH4)'
        else:
             ncm_sh_df['CO_SH4'] = ncm_sh_df['CO_SH4'].astype(int)
             top5 = top5.merge(
                 ncm_sh_df[['CO_SH4', 'NO_SH4_POR']],
                 left_on='SH4',
                 right_on='CO_SH4',
                 how='left'
             )
    except Exception as e:
        print(f"Erro ao mesclar dados top 5 com descrições NCM SH para CO_MUN {co_mun}: {e}")
        top5['NO_SH4_POR'] = 'Erro no Merge'

    top5['NO_SH4_POR'] = top5['NO_SH4_POR'].fillna('Descrição Desconhecida')
    top5['Produto'] = top5['NO_SH4_POR'].astype(str).str[:40] + ' (' + top5['SH4'].astype(str) + ')'
    top5['CO_MUN'] = co_mun

    columns_to_return = ['CO_MUN', 'SH4', 'Produto', calculated_metric_col]
    if metrica == 'Valor Agregado' and all(c in top5.columns for c in ['Total_FOB', 'Total_KG']):
        columns_to_return.extend(['Total_FOB', 'Total_KG'])

    return top5[columns_to_return]

In [ ]:
def criar_grafico_municipio_unico(dados, municipio_nome, ano, metrica):
    print(f"--- Gerando Gráfico: Município Único ---")
    print(f"  Município: {municipio_nome}")
    print(f"  Ano: {ano}")
    print(f"  Métrica: {metrica}")

    co_mun = get_co_mun(municipio_nome)
    if co_mun is None:
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, f"Erro: Código não encontrado para o município '{municipio_nome}'.",
                 ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    if ano not in dados or dados[ano] is None or dados[ano].empty:
        errmsg = f"Erro: Dados não disponíveis para o ano {ano}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    df_ano = dados[ano]
    top5_data = calcular_top_produtos(df_ano, co_mun, metrica)

    if top5_data.empty:
        infomsg = f"Sem dados de exportação Top 5 encontrados para:\n" \
                  f"Município: {municipio_nome} (Código: {co_mun})\n" \
                  f"Ano: {ano}\nMétrica: {metrica}"
        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}")
        plt.figure(figsize=(10, 4))
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off')
        plt.show()
        return

    coluna_plot = 'KG_LIQUIDO' if metrica == 'Quilogramas' else 'Valor_Agregado'
    if coluna_plot not in top5_data.columns:
        errmsg = f"Erro Interno: Coluna da métrica '{coluna_plot}' não encontrada nos dados calculados."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    eixo_x_label = 'Massa Líquida (Kg)' if metrica == 'Quilogramas' else 'Valor Agregado (USD/kg)'
    titulo = f'Top 5 Produtos Exportados - {municipio_nome} ({ano})'

    plt.figure(figsize=(12, 7))
    plot_data = top5_data.sort_values(coluna_plot, ascending=True)

    sns.barplot(data=plot_data, y='Produto', x=coluna_plot, color='cornflowerblue', orient='h')
    plt.title(titulo, fontsize=16, pad=15)
    plt.xlabel(eixo_x_label, fontsize=13)
    plt.ylabel('Produto (Descrição Resumida e Código SH4)', fontsize=13)
    plt.xticks(rotation=45, ha='right', fontsize=11)
    plt.yticks(fontsize=11)
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()
    print(f"  Plot displayed successfully for {municipio_nome}.")

def criar_grafico_top5_municipios(dados, ano, metrica):
    """Gera e exibe um gráfico de barras para os top 5 municípios exportadores de SP com base na métrica selecionada."""
    print(f"--- Gerando Gráfico: Top 5 Municípios SP ---")
    print(f"  Ano: {ano}")
    print(f"  Métrica: {metrica}")

    if ano not in dados or dados[ano] is None or dados[ano].empty:
        errmsg = f"Erro: Dados não disponíveis para o ano {ano}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    df_ano = dados[ano]

    if 'municipios_sp' not in globals() or municipios_sp.empty:
        errmsg = "Erro: Dados auxiliares de municípios ('municipios_sp') não disponíveis ou vazios.\n" \
                 "Não é possível adicionar nomes de municípios ao gráfico."
        log_msg = errmsg.replace('\n', ' ')
        print(f"  {log_msg}")
        plt.figure(figsize=(10, 3))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    df_agregado = pd.DataFrame()
    coluna_rank = None
    eixo_x_label = ''

    if metrica == 'Quilogramas':
        coluna_orig = 'KG_LIQUIDO'
        if coluna_orig not in df_ano.columns:
             print(f"  Erro: Coluna necessária '{coluna_orig}' não encontrada nos dados para {ano}.")
             return
        df_agregado = df_ano.groupby('CO_MUN')[coluna_orig].sum().reset_index()
        coluna_rank = coluna_orig
        eixo_x_label = 'Total Exportado (Kg)'

    elif metrica == 'Valor Agregado':
        if not all(col in df_ano.columns for col in ['VL_FOB', 'KG_LIQUIDO', 'CO_MUN']):
             print(f"  Erro: Colunas necessárias ('VL_FOB', 'KG_LIQUIDO', 'CO_MUN') não encontradas para {ano}.")
             return

        df_agregado_temp = df_ano.groupby('CO_MUN').agg(
            Total_FOB=('VL_FOB', 'sum'),
            Total_KG=('KG_LIQUIDO', 'sum')
        ).reset_index()

        epsilon = 1e-9
        df_agregado = df_agregado_temp[df_agregado_temp['Total_KG'] > epsilon].copy()

        if df_agregado.empty:
            infomsg = f"Sem municípios com exportações válidas (KG > 0) encontradas em {ano}\n" \
                      f"para calcular o Valor Agregado médio."
            log_msg = infomsg.replace('\n', ' ')
            print(f"  Info: {log_msg}")
            plt.figure(figsize=(10, 4))
            plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
            plt.axis('off')
            plt.show()
            return

        df_agregado['Valor_Agregado_Medio'] = df_agregado['Total_FOB'] / df_agregado['Total_KG']
        coluna_rank = 'Valor_Agregado_Medio'
        eixo_x_label = 'Valor Agregado Médio Municipal (USD/kg)'
    else:
        print(f"  Erro: Métrica desconhecida '{metrica}' para o gráfico Top 5 Municípios.")
        return

    if coluna_rank is None or coluna_rank not in df_agregado.columns:
         print(f"  Erro Interno: Coluna de ranking '{coluna_rank}' não encontrada nos dados agregados.")
         return

    top5_mun_codes = df_agregado.nlargest(5, coluna_rank)

    if top5_mun_codes.empty:
        infomsg = f"Não foi possível encontrar municípios para o ranking\n" \
                  f"Métrica: {metrica}\nAno: {ano}"
        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}")
        plt.figure(figsize=(10, 4))
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off')
        plt.show()
        return

    try:
         top5_mun_codes['CO_MUN'] = top5_mun_codes['CO_MUN'].astype(int)
         if 'CO_MUN_GEO' not in municipios_sp.columns:
             print("Erro: Coluna 'CO_MUN_GEO' ausente em municipios_sp. Não é possível mesclar nomes")
             top5_mun_data = top5_mun_codes.copy()
             top5_mun_data['NO_MUN'] = 'Erro no Merge (Sem CO_MUN_GEO)'
         else:
             municipios_sp['CO_MUN_GEO'] = municipios_sp['CO_MUN_GEO'].astype(int)
             top5_mun_data = top5_mun_codes.merge(
                 municipios_sp[['CO_MUN_GEO', 'NO_MUN']],
                 left_on='CO_MUN',
                 right_on='CO_MUN_GEO',
                 how='left'
             )
    except Exception as e:
         print(f"Erro ao mesclar códigos de municípios Top 5 com nomes: {e}")
         top5_mun_data = top5_mun_codes.copy()
         top5_mun_data['NO_MUN'] = 'Erro no Merge - Código: ' + top5_mun_data['CO_MUN'].astype(str)

    top5_mun_data['NO_MUN'] = top5_mun_data['NO_MUN'].fillna('Nome Desconhecido')

    plt.figure(figsize=(11, 7))

    plot_data = top5_mun_data.sort_values(coluna_rank, ascending=True)

    sns.barplot(data=plot_data, y='NO_MUN', x=coluna_rank, color='mediumseagreen', orient='h')

    plt.title(f'Top 5 Municípios Exportadores SP ({ano}) por {metrica}', fontsize=16, pad=15)
    plt.xlabel(eixo_x_label, fontsize=13)
    plt.ylabel('Município', fontsize=13)
    plt.xticks(rotation=45, ha='right', fontsize=11)
    plt.yticks(fontsize=11)
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()
    print(f"Gráfico exibido com sucesso para Top 5 Municípios.")

def criar_grafico_comparativo(dados, ano, metrica, municipio1_nome, municipio2_nome):

    print(f"--- Gerando Gráfico: Comparação de Municípios ---")
    print(f"  Município 1: {municipio1_nome}")
    print(f"  Município 2: {municipio2_nome}")
    print(f"  Ano: {ano}")
    print(f"  Métrica: {metrica}")

    if not municipio1_nome or not municipio2_nome or municipio1_nome == '-- N/A --' or municipio2_nome == '-- N/A --' or municipio1_nome == 'Select Municipality' or municipio2_nome == 'Select Municipality':
         errmsg = "Erro: Por favor, selecione dois municípios válidos para comparação."
         print(f"  {errmsg}")
         plt.figure(figsize=(10, 2))
         plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
         plt.axis('off')
         plt.show()
         return
    if municipio1_nome == municipio2_nome:
        errmsg = "Erro: Por favor, selecione dois municípios diferentes para comparação."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    co_mun1 = get_co_mun(municipio1_nome)
    co_mun2 = get_co_mun(municipio2_nome)
    if co_mun1 is None or co_mun2 is None:
        missing = []
        if co_mun1 is None: missing.append(f"'{municipio1_nome}'")
        if co_mun2 is None: missing.append(f"'{municipio2_nome}'")
        errmsg = f"Erro: Código não encontrado para: {', '.join(missing)}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    if ano not in dados or dados[ano] is None or dados[ano].empty:
        errmsg = f"Erro: Dados não disponíveis para o ano {ano}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    if 'ncm_sh_df' not in globals() or ncm_sh_df.empty:
        errmsg = "Erro: Dados auxiliares de produtos ('ncm_sh_df') não disponíveis.\n" \
                 "Não é possível adicionar descrições de produtos ao gráfico."
        log_msg = errmsg.replace('\n', ' ')
        print(f"  {log_msg}")
        plt.figure(figsize=(10, 3))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    df_ano = dados[ano]
    metrica_col_calc = 'KG_LIQUIDO' if metrica == 'Quilogramas' else 'Valor_Agregado'
    eixo_x_label = 'Massa Líquida (Kg)' if metrica == 'Quilogramas' else 'Valor Agregado (USD/kg)'

    def calculate_all_products(df_year, co_mun_sel, metric_name):
        df_mun_all = df_year[df_year['CO_MUN'] == co_mun_sel].copy()
        if df_mun_all.empty: return pd.DataFrame()

        metric_col = None
        agg_df = pd.DataFrame()

        if metric_name == 'Quilogramas':
             if 'KG_LIQUIDO' not in df_mun_all.columns: return pd.DataFrame()
             agg_df = df_mun_all.groupby('SH4')['KG_LIQUIDO'].sum().reset_index()
             metric_col = 'KG_LIQUIDO'
        elif metric_name == 'Valor Agregado':
             if not all(c in df_mun_all.columns for c in ['VL_FOB', 'KG_LIQUIDO']): return pd.DataFrame()
             agg_tmp = df_mun_all.groupby('SH4').agg(Total_FOB=('VL_FOB', 'sum'), Total_KG=('KG_LIQUIDO', 'sum')).reset_index()
             epsilon = 1e-9
             agg_df = agg_tmp[agg_tmp['Total_KG'] > epsilon].copy()
             if agg_df.empty: return pd.DataFrame()
             agg_df['Valor_Agregado'] = agg_df['Total_FOB'] / agg_df['Total_KG']
             metric_col = 'Valor_Agregado'
        else:
             print(f"Erro: Métrica desconhecida '{metric_name}' na função interna.")
             return pd.DataFrame()

        if metric_col and metric_col in agg_df.columns and not agg_df.empty:
             return agg_df[['SH4', metric_col]].copy()
        else:
             return pd.DataFrame()

    mun1_agg = calculate_all_products(df_ano, co_mun1, metrica)
    mun2_agg = calculate_all_products(df_ano, co_mun2, metrica)

    valid_mun1 = not mun1_agg.empty
    valid_mun2 = not mun2_agg.empty

    if not valid_mun1 and not valid_mun2:
        infomsg = f"Sem dados válidos para a métrica '{metrica}' ({ano})\n" \
                  f"encontrados para NENHUM dos municípios:\n" \
                  f"{municipio1_nome} ou {municipio2_nome}."

        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}")
        plt.figure(figsize=(10, 4))
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off')
        plt.show()
        return
    elif not valid_mun1:
         print(f"Aviso: Sem dados válidos encontrados para {municipio1_nome} (Métrica: {metrica}, Ano: {ano}). Comparação pode estar incompleta.")
    elif not valid_mun2:
         print(f"Aviso: Sem dados válidos encontrados para {municipio2_nome} (Métrica: {metrica}, Ano: {ano}). Comparação pode estar incompleta.")

    top5_sh4_mun1 = set(mun1_agg.nlargest(5, metrica_col_calc)['SH4']) if valid_mun1 else set()
    top5_sh4_mun2 = set(mun2_agg.nlargest(5, metrica_col_calc)['SH4']) if valid_mun2 else set()

    union_sh4 = top5_sh4_mun1.union(top5_sh4_mun2)

    if not union_sh4:
        infomsg = f"Nenhum produto comum no Top 5 encontrado para\n" \
                  f"{municipio1_nome} e {municipio2_nome} ({ano})\n" \
                  f"Métrica: {metrica}"
        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}")
        plt.figure(figsize=(10, 4))
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off')
        plt.show()
        return

    print(f"Comparando com base em {len(union_sh4)} produto(s) únicos do Top 5 (Códigos SH4: {union_sh4})")
    if valid_mun1:
        mun1_plot_data = mun1_agg[mun1_agg['SH4'].isin(union_sh4)].copy()
        mun1_plot_data['Municipio'] = municipio1_nome
    else:
        mun1_plot_data = pd.DataFrame({'SH4': list(union_sh4), metrica_col_calc: 0, 'Municipio': municipio1_nome})

    if valid_mun2:
        mun2_plot_data = mun2_agg[mun2_agg['SH4'].isin(union_sh4)].copy()
        mun2_plot_data['Municipio'] = municipio2_nome
    else:
        mun2_plot_data = pd.DataFrame({'SH4': list(union_sh4), metrica_col_calc: 0, 'Municipio': municipio2_nome})
    combined_data = pd.concat([mun1_plot_data, mun2_plot_data], ignore_index=True)

    try:
        combined_data['SH4'] = combined_data['SH4'].astype(int)
        if 'CO_SH4' not in ncm_sh_df.columns:
             print("Erro: Coluna 'CO_SH4' ausente em ncm_sh_df. Não é possível mesclar nomes de produtos.")
             combined_data['NO_SH4_POR'] = 'Erro no Merge (Sem CO_SH4)'
        else:
             ncm_sh_df['CO_SH4'] = ncm_sh_df['CO_SH4'].astype(int)
             combined_data = combined_data.merge(
                 ncm_sh_df[['CO_SH4', 'NO_SH4_POR']],
                 left_on='SH4',
                 right_on='CO_SH4',
                 how='left'
             )
    except Exception as e:
        print(f"Erro ao mesclar dados combinados com nomes NCM SH: {e}")
        combined_data['NO_SH4_POR'] = 'Erro no Merge'

    combined_data['NO_SH4_POR'] = combined_data['NO_SH4_POR'].fillna('Descrição Desconhecida')
    combined_data['Produto'] = combined_data['NO_SH4_POR'].astype(str).str[:40] + ' (' + combined_data['SH4'].astype(str) + ')'

    combined_data[metrica_col_calc] = combined_data[metrica_col_calc].fillna(0)

    plt.figure(figsize=(14, max(6, len(union_sh4) * 0.9)))

    palette = {'viridis': sns.color_palette("viridis", n_colors=2),
               'paired': sns.color_palette("Paired", n_colors=2)}
    plot_palette = palette['viridis']

    try:
        if not combined_data.empty and metrica_col_calc in combined_data.columns:
            product_order = combined_data.groupby('Produto')[metrica_col_calc].max().sort_values(ascending=False).index
            print(f"Ordem dos produtos no gráfico (decrescente por valor máximo): {list(product_order)}")
        else:
            product_order = None
    except Exception as e:
        print(f"Aviso: Não foi possível determinar a ordem dos produtos para o gráfico. Usando ordem padrão. Erro: {e}")
        product_order = None

    sns.barplot(
        data=combined_data,
        y='Produto',
        x=metrica_col_calc,
        hue='Municipio',
        orient='h',
        order=product_order,
        palette=plot_palette
    )

    plt.title(f'Comparação Top Produtos ({metrica}) - {municipio1_nome} vs {municipio2_nome} ({ano})', fontsize=16, pad=15)
    plt.xlabel(eixo_x_label, fontsize=13)
    plt.ylabel('Produto (Descrição Resumida e Código SH4)', fontsize=13)
    plt.xticks(rotation=45, ha='right', fontsize=11)
    plt.yticks(fontsize=11)
    plt.legend(title='Município', fontsize=11, title_fontsize=12, loc='lower right')
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()
    print(f"Gráfico de comparação exibido com sucesso.")

In [ ]:
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output_area:
            clear_output(wait=True)

            selected_ano = ano_dropdown.value
            selected_municipio = municipio_dropdown.value
            selected_metrica = metrica_dropdown.value

            if not selected_ano or not selected_municipio or \
               selected_municipio in ['-- N/A --', 'Selecionar Município']:
                errmsg = "Por favor, selecione um Ano e um Município válidos para visualizar o gráfico."
                print(f"Erro de Validação: {errmsg}")
                plt.figure(figsize=(10, 2))
                plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='orange')
                plt.axis('off')
                plt.show()
                return

            criar_grafico_municipio_unico(
                dados=dados_ano,
                municipio_nome=selected_municipio,
                ano=selected_ano,
                metrica=selected_metrica
            )
            print("Atualização de gráfico concluída.")

def on_top5_municipios_clicked(button_info):
     with output_area:
        clear_output(wait=True)
        print(">>> Evento: Botão 'Top 5 Municípios' clicado. Gerando gráfico")

        selected_ano = ano_dropdown.value
        selected_metrica = metrica_dropdown.value

        if not selected_ano:
             errmsg = "Por favor, selecione um Ano válido."
             print(f"! Erro de Validação: {errmsg}")
             plt.figure(figsize=(10, 2))
             plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='orange')
             plt.axis('off')
             plt.show()
             return

        criar_grafico_top5_municipios(
            dados=dados_ano,
            ano=selected_ano,
            metrica=selected_metrica
        )
        print("Atualização de gráfico concluída.")

def on_compare_clicked(button_info):
    with output_area:
        clear_output(wait=True)
        print("Evento: Botão 'Comparar Top 5 Produtos' clicado. Gerando gráfico de comparação...")

        selected_ano = ano_dropdown.value
        selected_metrica = metrica_dropdown.value
        selected_mun1 = municipio1_dropdown.value
        selected_mun2 = municipio2_dropdown.value

        criar_grafico_comparativo(
            dados=dados_ano,
            ano=selected_ano,
            metrica=selected_metrica,
            municipio1_nome=selected_mun1,
            municipio2_nome=selected_mun2
        )
        print("Atualização de gráfico concluída.")

In [ ]:
try:
    ano_dropdown.observe(on_dropdown_change, names='value')
    municipio_dropdown.observe(on_dropdown_change, names='value')
    metrica_dropdown.observe(on_dropdown_change, names='value')
    print("Observadores anexados a ano_dropdown, municipio_dropdown, metrica_dropdown.")
except NameError as e:
    print(f"Aviso: Não foi possível anexar observadores. Widget provavelmente não definido. Erro: {e}")

try:
    top5_municipios_button.on_click(on_top5_municipios_clicked)
    print("Manipulador de clique anexado a top5_municipios_button.")
except NameError as e:
     print(f"Aviso: Não foi possível anexar manipulador de clique a top5_municipios_button. Erro: {e}")

try:
    compare_button.on_click(on_compare_clicked)
    print("Manipulador de clique anexado a compare_button.")
except NameError as e:
     print(f"Aviso: Não foi possível anexar manipulador de clique a compare_button. Erro: {e}")

In [ ]:
common_filters = widgets.VBox([
    widgets.HTML("<b>Filtros Comuns:</b>", layout=widgets.Layout(margin='0 0 5px 0')),
    ano_dropdown,
    metrica_dropdown
], layout=widgets.Layout(margin='0 10px 0 0'))

controls_single = widgets.VBox([
    widgets.HTML("<b>Visualizar por Município:</b>", layout=widgets.Layout(margin='0 0 5px 0')),
    municipio_dropdown
], layout=widgets.Layout(margin='0 10px 0 0'))

controls_top_mun = widgets.VBox([
     widgets.HTML("<b>Visualizar Top Municípios (SP):</b>", layout=widgets.Layout(margin='15px 0 5px 0')),
     top5_municipios_button
])

controls_compare = widgets.VBox([
    widgets.HTML("<b>Comparar Municípios:</b>", layout=widgets.Layout(margin='15px 0 5px 0')),
    municipio1_dropdown,
    municipio2_dropdown,
    compare_button
])

left_column = widgets.VBox([common_filters, controls_single])
right_column = widgets.VBox([controls_top_mun, controls_compare])
dashboard_controls = widgets.HBox([left_column, right_column])

display(dashboard_controls)
print("--- Área de Saída (Gráficos aparecerão abaixo) ---")
display(output_area)

with output_area:
    print("--- Gerando Gráfico Inicial (baseado nos padrões) ---")

    initial_ano = ano_dropdown.value
    initial_municipio = municipio_dropdown.value
    initial_metrica = metrica_dropdown.value

    if initial_ano and initial_municipio and \
       initial_municipio not in ['-- N/A --', 'Select Municipality']:
        criar_grafico_municipio_unico(
            dados=dados_ano,
            municipio_nome=initial_municipio,
            ano=initial_ano,
            metrica=initial_metrica
        )
    else:
        warning_msg = "Não foi possível gerar o gráfico inicial.\n" \
                      "Verifique se os dados foram carregados corretamente (Células 3-5)\n" \
                      "e se há opções válidas selecionadas nos menus."
        log_msg = warning_msg.replace('\n', ' ')
        print(f"  ! Info: {log_msg}")
        plt.figure(figsize=(10, 3))
        plt.text(0.5, 0.5, warning_msg, ha='center', va='center', fontsize=12, color='orange')
        plt.axis('off')
        plt.show()